<a href="https://colab.research.google.com/github/svfrolov/YP-project/blob/main/%E2%84%961_%D0%98%D1%81%D1%81%D0%BB%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BD%D0%B0%D0%B4%D0%B5%D0%B6%D0%BD%D0%BE%D1%81%D1%82%D0%B8_%D0%B7%D0%B0%D0%B5%D0%BC%D1%89%D0%B8%D0%BA%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Данные имеют следующие столбцы
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

## План работы над проектом
1. Провести анализ пропусков, изменения данных и повторения значений.
2. Определить, какие некорректные строки требуется сохранить, а какие можно удалить.
3. Заполнить значения в пропущенных ячейках таблицы
4. Удалить дубликаты.
5. Вывести категории для проведения гипотез.
6. Составить выводы по каждому вопросу.
7. Составить общий вывод.

## Шаг 1. Откроем файл с данными и изучим общую информацию.

Подключаем библиотеки

In [5]:
import pandas as pd
data = pd.read_csv('https://drive.google.com/file/d/1mMBFG6Pe5LMRf-JVBq6zzEpnOGC4Q1em/view?usp=share_link')
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

Выведим первые 20 строчек датафрейма `data` на экран.

In [6]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем основную информацию о датафрейме с помощью метода `info()`.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

Выведите количество пропущенных значений для каждого столбца. Используйте комбинацию двух методов.

In [ ]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

заполним медианным доходом среди всех записей с тем же типом.

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.

In [ ]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [ ]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

Выведем перечень уникальных значений столбца `children`.

In [ ]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведите перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [ ]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.**

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены.

In [ ]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.

In [ ]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. Проверим остальные столбцы.

In [ ]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.

In [ ]:
data.duplicated().sum()

71

In [ ]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


Кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используем собственную функцию с именем `categorize_income()` и метод `apply()`.

In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [ ]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут нам правильно определить категорию.**

In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

In [ ]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


Данные очистили и разделили по категориям

### Шаг 3. Исследуем данные и ответим на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [ ]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
debt_s1 = data[(data['debt'] == 1)].count()
print('Просрочки и невозвраты составляют:', debt_s1['debt'])

Просрочки и невозвраты составляют: 1732


##### Выделим категории с детьми по их количеству в семье и сумме

In [ ]:
data_pivot2 = pd.pivot_table(data, index=['family_status_id', 'family_status'], \
                             columns='children', values=['debt'], aggfunc='sum')
data_pivot2

debt                              
children                                    0      1      2     3    4    5
family_status_id family_status                                             
0                женат / замужем        516.0  246.0  145.0  17.0  3.0  0.0
1                гражданский брак       229.0  118.0   30.0   8.0  0.0  0.0
2                вдовец / вдова          53.0    7.0    3.0   0.0  0.0  NaN
3                в разводе               55.0   21.0    7.0   1.0  0.0  NaN
4                Не женат / не замужем  210.0   52.0    9.0   1.0  1.0  NaN

##### Выясним доли просрочек?

In [ ]:
def doli_debt(column_list):
    smotr_table = data.groupby(column_list).agg({'debt':['sum', 'count']})
    smotr_table[('debt', '- - - %')] = smotr_table[('debt', 'sum')] / smotr_table[('debt', 'count')]*100
    return smotr_table

doli_debt(['children'])

debt                 
           sum  count   - - - %
children                       
0         1063  14091  7.543822
1          444   4808  9.234609
2          194   2052  9.454191
3           27    330  8.181818
4            4     41  9.756098
5            0      9  0.000000

In [ ]:
data.groupby('children')['debt'].mean()*100

children
0    7.543822
1    9.234609
2    9.454191
3    8.181818
4    9.756098
5    0.000000
Name: debt, dtype: float64

**Вывод:** 
в целом данные показывают, что при наличии детей от 1 до 4 просрочки присутствуют практически в равной степени, немного ниже в группе вообще без детей. А вот при наличии 5-ти детей просрочки отсутствуют вовсе. Если сравнивать группы без детей, а она в сумме больше всех остальных вместе взятых, с группой с детьми, то вероятность просрочек будет ниже в семьях без детей. Группа из 5-ти детей маленькая, но самая ответственная, учитывая данный момент, опираться на результат просрочек в такой маленькой выборке сложно. Но факт остается фактом, самые ответственные в группе из 5ти детей.  

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
data_pivot2 = pd.pivot_table(data, index=['family_status'], \
                             values=['debt'], aggfunc='count')
data_pivot2.sort_values('debt')

,debt
family_status,
вдовец / вдова,951
в разводе,1189
Не женат / не замужем,2796
гражданский брак,4134
женат / замужем,12261


In [ ]:
data.groupby('family_status')['debt'].mean()*100

family_status
Не женат / не замужем    9.763948
в разводе                7.064760
вдовец / вдова           6.624606
гражданский брак         9.313014
женат / замужем          7.560558
Name: debt, dtype: float64

In [ ]:
doli_debt(['family_status'])

debt                 
                       sum  count   - - - %
family_status                              
Не женат / не замужем  273   2796  9.763948
в разводе               84   1189  7.064760
вдовец / вдова          63    951  6.624606
гражданский брак       385   4134  9.313014
женат / замужем        927  12261  7.560558

**Вывод:** Просрочек больше у тех, кто состоит в отношениях, одиночки более дисциплинированы

**Исправил Вывод№2** Группа **не замужем и гражданский брак** достаточно крупные, но в них самые высокие просрочки **~9.5%**  **В группах - вразводе и вдова**, в два раза меньше количество, но и самые низкие просрочки порядка**6-7%**
**Семейные пары** - это самая многочисленная группа, больше всех вместе взятых и % просрочек один из самых нихких **~7,5%!** Семейные более ответственно подходят к возврату кредитов.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.

doli_debt(['total_income_category'])

debt                 
                        sum  count   - - - %
total_income_category                       
A                         2     25  8.000000
B                       354   5014  7.060231
C                      1353  15921  8.498210
D                        21    349  6.017192
E                         2     22  9.090909

**Вывод:** Группы с малым и большими доходами **А и E** слишком малы, да и процент просрочек выше - 8-9%. Более ответственно подходят к возврату группы категории **В и D** 7-6% соответственно, хотя количество клиентов в данных группах не слишком велико. **Группа С** самая большая и просрочки в ней составляют 8,5%, что не больше чем в остальных группах! В целом доход не сильно влияет на просрочки, скорее даже всех бошльше просрочек у богатых.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [ ]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
doli_debt(['purpose_category'])

debt                 
                          sum  count   - - - %
purpose_category                              
операции с автомобилем    400   4279  9.347978
операции с недвижимостью  780  10751  7.255139
получение образования     369   3988  9.252758
проведение свадьбы        183   2313  7.911803

**Вывод:** В целом - Всех меньше просрочек 183 в категории **проведение свадьбы** **7.9%**, но она самая малочисленная. Всех больше просрочек в категории **операции с недвижимостью** 780, но она самая большая и процент просрочек ниже остальных категорий **7,2%!** Также большие категории **операции с авто и образование**, но там процент просрочек выше чем в недвижимости **~9.3%**

#### 3.5 Приведем возможные причины появления пропусков в исходных данных.

Пропуски обнаружены в **days_employed и total_income** Вероятно пропуски произошли по вине человека при заполнении, а то что касается дохода, то вероятно его могли намеренно не указывать. Интересная деталь, что пропусков одинаковое количество в обеих категориях **2174**, вероятно они не были указаны.



**Вероятно пропуски произошли по вине человека при заполнении, а то что касается дохода, то вероятно его могли намеренно не указывать** - - но в таком случае заемщик не получил бы кредит! А он его получил по факту, так как соответствующие строки находятся в одной из таблиц данных нашего банка.

---
Действительно, в наших данных пропуски в столбцах "трудовой стаж" и "ежемесячный доход" абсолютно идентичны. Количество строк с отсутствующими данными в обоих указанных столбцах совпадает и равно 2174. Маловероятно, что специалисты по выдаче кредитов, заполнив все без исключения ячейки других столбцов по каждому кредитополучателю, оставляли незаполненными именно два показателя, и сделали это ровно 2174 раза в каждом столбце. Даже не проводя проверку, во всех ли случаях незаполненные ячейки находятся в одной и то же строке с высокой долей вероятности можно утверждать, что произошла техническая ошибка при формировании таблицы с исходными данными (либо некий сбой во время выгрузки данных на анализ, либо при объединении данных в единый файл, который также должен был передаваться на анализ, либо эти данные не собирались). 

На пользу основной версии об «объединении», как причине проблем с данными работает вот ещё какой аргумент: пропуски в столбцах 'total_income' и 'days_employed' соответствуют значениям 'пенсионер', 'госслужащий', 'компаньон', 'сотрудник' и 'предприниматель' в столбце 'income_type'. Однако в 'income_type' также присутствуют значения ‘студент’, ‘безработный’ и ‘в декрете’, в которых пропусков нет вообще. На основании этого можно предположить, что исходная таблица была объединена с другим датасетом, содержавшим сведения о категориях 'пенсионер', 'госслужащий', 'компаньон', 'сотрудник' и 'предприниматель', но без сведений о доходе и трудовом стаже. В результате при объединении таблиц столбцы 'total_income' и 'days_employed' были заполнены значением NaN.

#### 3.6 Объясним, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

В случае, если в данных наблюдаются "выбросы", то даже небольшое их количество может сильно "перетягивать" среднее значение в большую или меньшую сторону. Медиана зачастую даёт более правдивую картину. В общем случае, в реальных "живых" проектах я рекомендую в каждом случае смотреть на то, как распределены данные.

### Шаг 4: общий вывод.

Провели работу по очистке данных, удалили пропуски, обработали аномалии и дубликаты, добавили категории для удобства анализа. В целом выявили определённые закономерности по просрочкам в разных категориях.

**Проанализированы данные** В результате мы удалили пропуски и заполнили их медианными значениями. Удалили дубликаты.

**Общий вывод следующий:**
Наличие детей в равной степени одинаково влияет на возврат кредита, просрочка порядка **~9%**
Семьи без детей достаточно большая группа да и процент просрочек ниже **7,5%**
Исключением явилась небольшая группа из **5** детей, просрочек нет вообще **0%**, но в связи с малочисленностью опираться на данную выборку сложно

Группа **не замужем и гражданский брак** достаточно крупные, но в них самые высокие просрочки **~9.5%** 
В группах - **вразводе и вдова**, в два раза меньше количество, но и самые низкие просрочки порядка **6-7%** 
**Семейные пары** - это самая многочисленная группа, больше всех вместе взятых и **%** просрочек один из самых нихких **~7,5%**

**Семейные более ответственно подходят к возврату кредитов.**

Группы с малыми доходами сравнительно небольшие, верноятно не могут обеспечить стабильный фин поток и не берут кредиты **8%**
Группы большими доходами не слишком видимо нуждаются в кредитах, потому и слишком малы, но процент просрочек выше **9%**. 
Более ответственно подходят к возврату группы категории В и D 7-6% соответственно, хотя количество клиентов в данных группах не слишком велико. Группа С самая большая и просрочки в ней составляют 8,5%, что не больше чем в остальных группах! 
**В целом доход не сильно влияет на просрочки, скорее даже всех бошльше просрочек у богатых.**

В **категориях автомоБили, оБразование, недвиЖимость**, самая Большая группа это недвижимость просрочек в данной групе меньше всех **7,2%** автомобили и образование**9,5%** свадьбы **7,9*%**, но это самая малочисленная группа

**В целом по группах процент просрочек относительно схожий, предпологаю, что на просрочку оказывает влияние в первую очередь семейное положение, затем наличие детей, следом доход - если он позволяет взять кредит и на четвертом месте цель.**